In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras 
import pandas as pd 
import os 
from sklearn.model_selection import StratifiedKFold 
import gc 
import random
import pickle 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import albumentations as A 
# 统计代码运行的时间
from time import *
begin_time = time()

SEED = 2020
from tensorflow import random
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
random.set_seed(SEED)

data_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/data/'
model_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/model/'
npy_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/npy_file/'
pseudo_path = '/content/drive/My Drive/kesci_JZB/交子杯算法赛道—渣渣炼丹师队/code/pseudo_labels/'

In [6]:
def My_acc_combo(y, y_pred, mode): 
    if mode== 'behavior':
        # 将行为ID转为编码
        code_y, code_y_pred = mapping[y], mapping[y_pred]
        if code_y == code_y_pred: # 编码完全相同得分1.0
            return 1.0
        elif code_y.split("_")[0] == code_y_pred.split("_")[0]: # 编码仅字母部分相同得分1.0/7
            return 1.0/7
        elif code_y.split("_")[1] == code_y_pred.split("_")[1]: # 编码仅数字部分相同得分1.0/3
            return 1.0/3
        else:
            return 0.0

In [7]:
# 特征列名称
src_names = ['acc_x', 'acc_y', 'acc_z', 'acc_xg', 'acc_yg', 'acc_zg', 'acc', 'acc_g']

def handle_features(data):
    data.drop(columns = ['time_point'], inplace=True)

    data['acc'] = (data.acc_x ** 2 + data.acc_y ** 2 + data.acc_z ** 2) ** 0.5
    data['acc_g'] = (data.acc_xg ** 2 + data.acc_yg ** 2 + data.acc_zg ** 2) ** 0.5

    return data

# 构造numpy特征矩阵
def handle_mats(grouped_data):
    mats = [i.values for i in grouped_data] # i.values将i(每一个时序片段，但是长度不一)转化为数组
    # padding 填充 或者 重采样也可以。
    for i in range(len(mats)): 
      padding_times = 61 - mats[i].shape[0] # 片段长度最多61，剩下的填充
      for j in range(padding_times):
        mats[i] = np.append(mats[i], [[0 for _ in range(mats[i].shape[1])]], axis=0) # _ 占位符，表示不在意变量的值，只是用于循环遍历n次
        # np.append(arr, values, axis=None) 为原始array添加一些values，这里是添加一行(1,mats[i].shape[1])元素全为0的二维数组
        # 当arr的维数为2，axis=0表示沿着行方向添加values；axis=1表示沿着列方向添加values

    mats_padded = np.zeros([len(mats), 61, mats[0].shape[1]]) # 每个时序片段长度为61
    for i in range(len(mats)):
      mats_padded[i] = mats[i] 

    return mats_padded # 每个时序片段相同大小的三维数组

def get_train_data(use_scaler=True, shuffle=False, pseudo_labels_file=None):
    df = pd.read_csv(data_path + "sensor_train.csv")

    # 简单拼接伪标签
    if pseudo_labels_file != None:
      df = df.append(pd.read_csv(pseudo_labels_file)) # 测试集中，主模型预测概率最高(＞0.95)的数据，作为已知的数据标签
      # df1.append(df2)最简单的拼接数据的方式，注意行索引为各自对应的行索引
    data = handle_features(df)

    # 标准化，并将统计值保存
    if use_scaler:
      scaler = StandardScaler()
      scaler.fit(data[src_names].values) # 用于计算训练数据的均值和方差，然后保存下来，用测试集的均值和方差来转换训练数据  scaler里面存的有计算出来训练集的均值和方差
      with open(pseudo_path + 'D_CNNscaler.pkl', 'wb') as f: # wb:以二进制格式打开一个文件只用于写入。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。
        pickle.dump(scaler, f) 
        # pickle.dumps(obj, file) 序列化对象，将对象obj保存到文件file中去。
        # file对象必须有write()接口，file可以是一个以'w'打开的文件或者是一个String对象，也可以是任何可以实现write()接口的对象。
      data[src_names] = scaler.transform(data[src_names].values) # 这一步是用训练数据的均值和方差来转换训练数据，使训练数据标准化

    ######## 注意：在预测的时候，要用上面训练集fit得到的scaler中的均值和方差来对 "测试集数据" 进行标准化处理，即transform ########

    grouped_data = [i.drop(columns='fragment_id') for _, i in data.groupby('fragment_id')]
    train_labels = np.array([int(i.iloc[0]['behavior_id']) for i in grouped_data]) # i.iloc[0]['behavior_id']第一行 str --> int
    for i in range(len(grouped_data)): 
      grouped_data[i].drop(columns='behavior_id', inplace=True)
    train_data = handle_mats(grouped_data)
    
    if shuffle:
      index = [i for i in range(len(train_labels))]
      np.random.seed(SEED)
      np.random.shuffle(index)

      train_data = train_data[index]
      train_labels = train_labels[index]

    return train_data, train_labels

def get_test_data(use_scaler=True):
    FILE_NAME = data_path + "sensor_test.csv"
    FILE_NAME1 = data_path + "sensor_train.csv"
    data = handle_features(pd.read_csv(FILE_NAME))
    train_ori = handle_features( pd.read_csv(FILE_NAME1)[['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg', 'acc_yg', 'acc_zg']] ) # 没有标签

    if use_scaler:
      with open(pseudo_path + 'D_CNNscaler.pkl', 'rb') as f: # rb:以二进制格式打开一个文件用于只读。
        scaler = pickle.load(f) # 反序列化对象，将文件中的数据解析为一个python对象。file对象需要有read()接口和readline()接口。
      data[src_names] = scaler.transform(data[src_names].values)
      train_ori[src_names] = scaler.transform(train_ori[src_names].values)

    grouped_data = [i.drop(columns='fragment_id') for _, i in data.groupby('fragment_id')]
    grouped_data1 = [i.drop(columns='fragment_id') for _, i in train_ori.groupby('fragment_id')]
    # data.groupby('fragment_id')得到(k,dataframe(k))，所以，i得到的是每个fragment_id所对应的dataframe
    return [handle_mats(grouped_data), handle_mats(grouped_data1)]

def get_train_test_data(use_scaler=True, shuffle=True, pseudo_labels_file=None):
    train_data, train_lables = get_train_data(use_scaler, shuffle=False, pseudo_labels_file=pseudo_labels_file)
    test_data, train_ori = get_test_data(use_scaler) 
    return train_data, train_lables, test_data, train_ori

In [8]:
kfcv_seed = 2020  
data_enhance_method = [] 
mapping = { 0:'A_0', 1:'A_1', 2:'A_2', 3:'A_3', 
        4:'D_4', 5:'A_5', 6:'B_1', 7:'B_5', 
        8:'B_2', 9:'B_3', 10:'B_0', 11:'A_6', 
        12:'C_1', 13:'C_3', 14:'C_0', 15:'B_6', 
        16:'C_2', 17:'C_5', 18:'C_6' }
reversed_mapping = { value: key for key, value in mapping.items() } # 反字典


def set_data_enhance(val):
  if not isinstance(val, list): # isinstance()函数来判断val是否是list，若是，返回True，否则，返回False
    val = [val] # 非列表的转化为列表
  global data_enhance_method # 设置可以在函数中调用的全局变量 
  data_enhance_method = val

## 获取行为对应的场景和动作编码
def decode_label(label_code):
  str = mapping[label_code] # 传入key，得到key值(字符串)
  scene_code = ord(str.split('_')[0]) - ord('A') # 比如：str.split('_')[0]='A'，则ord('A')=65 ord()函数以一个字符串作为参数，返回对应的ASCII数值
  action_code = ord(str.split('_')[1]) - ord('0') # ord('0')=48
  return scene_code, action_code 

## 交叉评估 
def kfcv_evaluate(model_name, x, y):
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=kfcv_seed)
    evals = {'loss':0.0, 'accuracy':0.0}

    for kfold_, (train, val) in enumerate(kfold.split(x, np.argmax(y, axis=-1))): # train、val为训练和验证集索引

        print('Processing fold: %d (%d, %d)' % (kfold_, len(train), len(val)))
        model = keras.models.load_model(model_path + '%s/part_%d.h5' % (model_name, kfold_)) # 加载每一折训练的最优模型
        loss, acc = model.evaluate(x[val], y[val]) # 返回的是损失值和选定的指标值(精度)
        evals['loss'] += loss / k # 交叉验证的损失取平均
        evals['accuracy'] += acc / k # # 交叉验证的精度取平均

    return evals 

## 交叉预测
def kfcv_predict(model_name, inputs):
  # path = model_path + model_name + '/'
  models = []
  for i in range(k):
    # models.append(keras.models.load_model(path + 'part_%d.h5' % i)) 
    models.append(keras.models.load_model(model_path + '%s/part_%d.h5' % (model_name, i)))
  print('%s loaded.' % model_name) 
  result = [] 
  for j in range(k):
    result.append(models[j].predict(inputs)) # inputs为测试集

  print('result got')
  result = sum(result) / k # 交叉验证的预测结果取平均，类似单模stacking
  return result 


## 白噪声(noise)、混合(mixup)、黑色正方块(cutout)
def data_enhance(method, train_data, train_labels): # train_labels：(?, 19)

    if method == 'noise':
      noise = train_data + np.random.normal(loc = 0, scale = 0.1, size = train_data.shape) 
      # 均值、标准差 正态分布的标准差，对应分布的宽度，scale越大，正态分布的曲线越矮胖，scale越小，曲线越高瘦。
      return noise, train_labels 

    elif method == 'cutout': 
      cutout = np.zeros((train_data.shape[0], train_data.shape[1], train_data.shape[2]))
      for i in range(train_data.shape[0]):  
        transform = A.Cutout(num_holes=8, max_h_size=1, max_w_size=1, always_apply=False, p=0.5) # Cutout 剪下8个正方形的小黑块
        cutout[i, :, :] = transform(image = train_data[i, :, :])['image'] # 生成一个裁剪之后的图片
      train_data = np.vstack((train_data, cutout)) # 多维数组垂直堆叠
      train_labels = np.vstack((train_labels, train_labels)) # y为标签，一维数组，可以水平堆叠
      return train_data, train_labels 

    elif method == 'mixup': # 几乎无额外计算开销的情况下稳定提升1个百分点的图像分类精度
      index = [i for i in range(len(train_labels))] 
      np.random.shuffle(index) # 打乱索引 

      x_mixup = np.zeros(train_data.shape)
      y_mixup = np.zeros(train_labels.shape)

      for i in range(len(train_labels)):
        x1 = train_data[i] # 每一个图片
        x2 = train_data[index[i]] # 随机打乱的索引所对应的图片
        y1 = train_labels[i] 
        y2 = train_labels[index[i]]

        factor = np.random.beta(0.2, 0.2) # 得到一个趋近于0或者1的概率 https://zhuanlan.zhihu.com/p/24555092
        # beta分布就是抛硬币a次正，b次反后，硬币正面概率的分布，当a,b都小时，概率要么趋近于0，要么趋近于1
        x_mixup[i] = x1 * factor + x2 * (1 - factor) # 把混合之后的图片作为第i个图片
        y_mixup[i] = y1 * factor + y2 * (1 - factor) # 标签也是

      return x_mixup, y_mixup 


def shuffle(data, labels, seed=None):
    index = [i for i in range(len(labels))]
    if seed != None:
      np.random.seed(seed) 
    np.random.shuffle(index) # 打乱索引列表
    return data[index], labels[index]
 
## 交叉拟合
def kfcv_fit(builder, x, y, epochs, checkpoint_path, verbose=2, batch_size=64): # checkpoint_path --> 传入model_path
    kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=kfcv_seed)
    histories = [] # 历史记录
    evals = [] # 评估标准

    if checkpoint_path[len(checkpoint_path) - 1] != '/': # 路径字符串最后一个字符如果不是 '/'
      checkpoint_path += '/' # 拼接

    for i in range(k):
      if os.path.exists(checkpoint_path + 'part_%d.h5' % i): # 如果路径path存在，返回True；如果路径path不存在，返回False。
        os.remove(checkpoint_path + 'part_%d.h5' % i) # os.remove()方法用于删除指定路径的文件。

    for index, (train, val) in enumerate(kfold.split(x, np.argmax(y, axis=-1))):
        print('Processing fold: %d (%d, %d)' % (index, len(train), len(val)))
        model = builder()

        x_train = x[train] # 分折训练的数据
        y_train = y[train]

        if len(data_enhance_method) > 0:
          x_train_copy = np.copy(x_train)
          y_train_copy = np.copy(y_train)
          for method in data_enhance_method: 
            x_, y_ = data_enhance(method, x_train_copy, y_train_copy)
            x_train = np.r_[x_train, x_] # np.r_ 按行拼接矩阵，要求列数相同 a = [[1,2,3],[7,8,9]] b = [[4,5,6],[1,2,3]] np.r_[a,b] --> (4,3)
            y_train = np.r_[y_train, y_] 
          x_train, y_train = shuffle(x_train, y_train) # 打乱数据
          print('Data enhanced (%s) => %d' % (' '.join(data_enhance_method), len(x_train)))
        # ' '.join(data_enhance_method)把list里的字符串元素用' '拼接成一个字符串，比如：data_enhance_method=['noise','cutout'] --> 'noise cutout'
        plateau = ReduceLROnPlateau(monitor = 'val_acc', verbose = 0, mode = 'max', factor = 0.1, patience = 8) # 'val_acc_combo'
        early_stopping = EarlyStopping(monitor = 'val_acc', verbose = 0, mode = 'max', patience = 20) # 防止过拟合
        checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path + 'part_%d.h5' % index,
                                 monitor='val_accuracy',
                                 verbose=0,
                                 mode='max',
                                 save_best_only=True)

        h = model.fit(x_train, 
                y_train,
                epochs=epochs,
                verbose=verbose,
                validation_data=(x[val], y[val]),
                callbacks=[plateau, early_stopping, checkpoint],
                batch_size=batch_size,
                shuffle=True) 
        evals.append(model.evaluate(x=x[val], y=y[val])) # model.evaluate 返回一个列表
        histories.append(h) 
        del model
        gc.collect() 
    return histories, evals 


def save_results(path, output): 
    print('saving...')
    df_r = pd.DataFrame(columns=['fragment_id', 'behavior_id'])
    for i in range(len(output)):
      behavior_id = output[i] # 遍历预测的行为列表
      df_r = df_r.append({'fragment_id': i, 'behavior_id': behavior_id}, ignore_index=True) 
      # 向dataframe对象中添加新的行，通过设置ignore_index=True来避免index出现重复的情况。
    df_r.to_csv(path, index=False)


def infer(model_name, inputs, csv_output): # csv_output --> data_path
    output = np.argmax(kfcv_predict(model_name, inputs), axis=-1) 
    save_results(csv_output, output) 
    print('- END -')
    print('Your file locates at %s' % csv_output)

In [9]:
def BLOCK(seq, filters, kernal_size): 
    cnn = keras.layers.Conv1D(filters, 1, padding='SAME', activation='relu')(seq) # (?, 61, 8) --> (?, 61, 128)
    cnn = keras.layers.LayerNormalization()(cnn) # LN：取的是同一个样本的不同通道做归一化，根据样本的特征数做归一化 BN：取不同样本的同一个通道的特征做归一化 
    # 参考：https://zhuanlan.zhihu.com/p/54530247 
    cnn = keras.layers.Conv1D(filters, kernal_size, padding='SAME', activation='relu')(cnn) # (?, 61, 128) --> (?, 61, 128)
    cnn = keras.layers.LayerNormalization()(cnn) # 输入下一层之前进行层归一化

    cnn = keras.layers.Conv1D(filters, 1, padding='SAME', activation='relu')(cnn) # (?, 61, 128) --> (?, 61, 128)
    cnn = keras.layers.LayerNormalization()(cnn)

    seq = keras.layers.Conv1D(filters, 1)(seq)  
    seq = keras.layers.Add()([seq, cnn]) # 该层接收一个相同shape列表张量，并返回它们的和，shape不变。 高层特征与底层特征进行融合
    # [(?, 61, 128), (?, 61, 128)] --> (?, 61, 128)
    return seq

def BLOCK2(seq, filters=128, kernal_size=3):
    seq = BLOCK(seq, filters, kernal_size) 
    seq = keras.layers.AveragePooling1D(2)(seq) # (?, 61, 128) --> (?, 30, 128)
    seq = keras.layers.SpatialDropout1D(0.3)(seq) 
    # 它断开的是整个1D特征图，而不是单个神经元。
    # 如果一张特征图的相邻像素之间有很强的相关性（通常发生在低层的卷积层中），那么普通的dropout无法正则化其输出，否则就会导致明显的学习率下降。
    # 这种情况下，SpatialDropout1D能够帮助提高特征图之间的独立性，应该用其取代普通的Dropout
    seq = BLOCK(seq, filters//2, kernal_size) # 两个斜杠即双斜杠（//）表示地板除，即先做除法（/），然后向下取整（floor），比如：3 // 2 = 1
    # [(?, 30, 64), (?, 30, 64)] --> (?, 30, 64)
    seq = keras.layers.GlobalAveragePooling1D()(seq) # (?, 30, 64) --> (?, 64)
    return seq

def ComplexConv1D(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape[1:]) # input_shape --> (10866, 61, 8) input_shape[1:] --> (61, 8)
    seq_3 = BLOCK2(inputs, kernal_size=3) # (?, 30, 64) --> (?, 64)
    seq_5 = BLOCK2(inputs, kernal_size=3) # (?, 30, 64) --> (?, 64)
    seq_7 = BLOCK2(inputs, kernal_size=3) # (?, 30, 64) --> (?, 64)
    seq = keras.layers.concatenate([seq_3, seq_5, seq_7]) # [(?, 64), (?, 64), (?, 64)] --> (?, 192)
    ## 用CNN1D提取时序特征之后，再用普通的隐层进行拟合
    seq = keras.layers.Dense(512, activation='relu')(seq) # (?, 192) --> (?, 512) 
    seq = keras.layers.Dropout(0.3)(seq) 
    seq = keras.layers.Dense(128, activation='relu')(seq) # (?, 512) --> (?, 128) 
    seq = keras.layers.Dropout(0.3)(seq)
    outputs = keras.layers.Dense(num_classes, activation='softmax')(seq) # (?, 128) --> (?, 19)  

    return keras.models.Model(inputs=[inputs], outputs=[outputs])

In [11]:
# 导入主模型挑选的pseudo labels，这么做的目的是把从主模型预测准确率较高的测试集样本提取出来，跟原训练数据进行拼接，作为新模型的训练数据
train_data, train_labels, test_data, train_ori = get_train_test_data(pseudo_labels_file = pseudo_path + 'pseudo_labels.csv') # array
# train_data --> (10866, 61, 8) train_labels --> (10866,) test_data --> (7500, 61, 8) train_ori --> (7292, 61, 8)

In [12]:
cv = 20
la = pd.read_csv(data_path + "sensor_train.csv")
y_train = la.groupby('fragment_id')['behavior_id'].min()
checkpoint_path = model_path

for i in range(cv): # 5折
  if os.path.exists(checkpoint_path + 'part_%d.h5' % i):
    os.remove(checkpoint_path + 'part_%d.h5' % i)
## 原训练数据及标签 
x = train_data[ : 7292] 
y = train_labels[ : 7292] # (7292, )
y_ = to_categorical(y, num_classes=19) # (7292, 19) 
## 由主模型得到的伪训练数据及标签 
wei_train_x = train_data[7292 : ]
wei_label_y = train_labels[7292 : ]
wei_label_y_ = to_categorical(wei_label_y, num_classes=19)

proba_t = np.zeros((7500, 19))
valid = np.zeros((7292, 19)) 

In [ ]:
cv = 20
la = pd.read_csv(data_path + "sensor_train.csv")
y_train = la.groupby('fragment_id')['behavior_id'].min()
checkpoint_path = model_path

for i in range(cv): # 5折
  if os.path.exists(checkpoint_path + 'part_%d.h5' % i):
    os.remove(checkpoint_path + 'part_%d.h5' % i)
## 原训练数据及标签 
x = train_data[ : 7292] 
y = train_labels[ : 7292] # (7292, )
y_ = to_categorical(y, num_classes=19) # (7292, 19) 
## 由主模型得到的伪训练数据及标签 
wei_train_x = train_data[7292 : ]
wei_label_y = train_labels[7292 : ]
wei_label_y_ = to_categorical(wei_label_y, num_classes=19)

proba_t = np.zeros((7500, 19))
valid = np.zeros((7292, 19)) 

## 为伪标签创建20折，然后训练时分配下去
valid_fold = [] # 每个元素为一个索引list 比如：(179,)
kfold1 = StratifiedKFold(n_splits=cv, shuffle=True, random_state=2020) 
for index, (train, val) in enumerate(kfold1.split(wei_train_x, wei_label_y)):
  valid_fold.append(val) 

## 原数据 20折交叉验证 验证trick是否有提升
kfold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=2020)  
## 总结：对shuffle=True设置random_state=2020之后，每次结果都是固定的，它只会随着不同的种子而改变。
## 如果想要每次打乱的结果都是不一样的，只需要设置shuffle=True，而不需要设置随机种子
## 因此，验证trick是否有提升的时候，必须要在设置固定随机种子的基础上再进行对比。
for index, (train, val) in enumerate(kfold.split(x, y)):
  print('Processing fold: %d (%d, %d)' % (index, len(train), len(val))) # (6927, 365)
  model = ComplexConv1D(train_data.shape, 19) 
  model.compile(optimizer='Nadam', loss=tf.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=['accuracy']) 
  ## 无数据增强 
  # x_train, y_train = x[train], y_[train]
  ########################### 训练数据增强 ###########################
  ## 噪音增强
  # x_train, y_train = data_enhance('noise', x[train], y_[train])
  ## mixup增强：几乎无额外计算开销的情况下稳定提升1个百分点的图像分类精度
  # x_train, y_train = data_enhance('mixup', x[train], y_[train])
  ## cutout增强：数据变成原来的两倍

  x_train, y_train = data_enhance('cutout', x[train], y_[train]) # x_train：(13854, 61, 8) y_train：(13854, 19)
  
  ## 再加上半监督增强
  # x_train = np.vstack([x_train, wei_train_x[valid_fold[index]]]) # 6927/13854 + 179 = 7106/14033 每一次只增加伪标签数据的一部分(验证集大小) 
  # y_train = np.vstack([y_train, wei_label_y_[valid_fold[index]]]) 
  x_train = np.vstack([x_train, wei_train_x]) # 6927/13854 + 3474 = 10466/17328
  y_train = np.vstack([y_train, wei_label_y_]) 
  
  # 原始数据(7292) + 半监督增强(179) 0.86301  
  # cutout增强数据(7292 * 2) + 半监督增强(179) 0.87671
  # 原始数据(7292) + 半监督增强(3474) 0.87671 
  # 噪音增强数据(7292) + 半监督增强(3474) 0.84384
  # mixup增强数据(7292) + 半监督增强(3474) 0.86301
  # cutout增强数据(7292 * 2) + 半监督增强(3474) 0.89041 

  graph_path = model_path + 'ComplexConv1D_model.png'
  plot_model(model, to_file = graph_path, show_shapes = True)  # 绘制模型图
  
  early_stopping = EarlyStopping(monitor='val_accuracy', verbose=0, mode='max', patience=20)
  checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path + 'part_%d.h5' % index, monitor='val_accuracy', verbose=0, mode='max', save_best_only=True)
  plateau = ReduceLROnPlateau(monitor="val_accuracy", verbose=0, mode='max', factor=0.1, patience=8) 
  h = model.fit(x=x_train, 
          y=y_train, 
          epochs=500,  
          verbose=1,  
          validation_data=(x[val], y_[val]),
          callbacks=[checkpoint,early_stopping,plateau], 
          batch_size=64,
          shuffle=True)
  
  model.load_weights(checkpoint_path + 'part_%d.h5' % index)

  # 单模stacking
  valid[val] += model.predict(x[val])
  proba_t += model.predict(test_data) / cv

  # 计算验证集精确率
  val_ = model.predict(x[val])
  print('准确率得分：', round(accuracy_score(y[val], np.argmax(val_, axis=1)), 5))

  del model
  gc.collect()

# 计算执行时间 
end_time = time()
run_time = (end_time - begin_time) / 60
print('该循环程序运行时间：', run_time, '分钟') 

np.save(npy_path + 'oneD_CNN_valid', valid)
np.save(npy_path + 'oneD_CNN_test', proba_t)

Processing fold: 0 (6927, 365)
Epoch 1/500
273/273 [==============================] - 19s 71ms/step - loss: 2.0021 - accuracy: 0.4331 - val_loss: 1.7111 - val_accuracy: 0.5041
Epoch 2/500
273/273 [==============================] - 19s 68ms/step - loss: 1.6432 - accuracy: 0.5702 - val_loss: 1.5115 - val_accuracy: 0.6055
Epoch 3/500
273/273 [==============================] - 19s 68ms/step - loss: 1.4840 - accuracy: 0.6406 - val_loss: 1.4001 - val_accuracy: 0.6822
Epoch 4/500
273/273 [==============================] - 19s 68ms/step - loss: 1.3825 - accuracy: 0.6885 - val_loss: 1.3365 - val_accuracy: 0.7288
Epoch 5/500
273/273 [==============================] - 18s 65ms/step - loss: 1.3147 - accuracy: 0.7175 - val_loss: 1.2798 - val_accuracy: 0.7014
Epoch 6/500
273/273 [==============================] - 18s 65ms/step - loss: 1.2492 - accuracy: 0.7503 - val_loss: 1.2503 - val_accuracy: 0.7205
Epoch 7/500
273/273 [==============================] - 19s 68ms/step - loss: 1.1998 - accuracy: 0.7